In [2]:
import json
import tqdm
import numpy as np
import pandas as pd

# import pymorphy2
# from wiki_ru_wordnet import WikiWordnet

# morph = pymorphy2.MorphAnalyzer()
# wiki = WikiWordnet()

In [ ]:
c_d={}
if False:
    with open('TlkPersonaChatRus/tolokapersonachat.jsonl', 'r') as inp:
        for line in tqdm.tqdm(inp):
            line = json.loads(line)
            for k in line:
                if k=='responce':
                    for word in line[k].split():
                        word = [l for l in word if l.isalpha()]
                        word = ''.join(word)
                        word = morph.parse(word)[0].normal_form
                        try:
                            c_d[word]+=1
                        except KeyError:
                            c_d[word]=1
                elif k=='label':
                    pass
                else:
                    for sent in line[k]:
                        for word in sent.split():
                            word = [l for l in word if l.isalpha()]
                            word = ''.join(word)
                            word = morph.parse(word)[0].normal_form
                            word = morph.parse(word)[0].normal_form
                            try:
                                c_d[word]+=1
                            except KeyError:
                                c_d[word]=1
            with open('TlkPersonaChatRus/count_dict.json', 'w') as out:
                out.write(json.dumps(c_d, ensure_ascii=False))
                
print(c_d)

In [6]:
personas = pd.read_csv('TlkPersonaChatRus/profiles.tsv', sep='\t', index_col=False)
personas

,characteristic_1,characteristic_2,characteristic_3,characteristic_4,characteristic_5
0,У меня любимая работа.,Я уважаю людей.,У меня есть животное.,У меня хороший друг.,Я люблю кофе.
1,Я работаю учителем,У меня есть собака,Я люблю петь,Я живу сама,Я люблю цветы
2,Я купила дом,Я бегаю по утрам,Я работаю на работе,Я поеду в отпуск,Я люблю арбуз
3,я врач и женат,у меня трое детей,не люблю свою работу,нравиться ездить на велосипеде,люблю пиво
4,Я школьница.,Я ещё учусь.,Но я мечтаю работать.,Я обожаю родителей.,И не люблю учиться.
...,...,...,...,...,...
1500,Я работаю на любимой работе,Я люблю собак,Смотрю советские фильмы,Играю в футбол,У меня двое дитей
1501,Я блондинка,Я бизнес леди,Я люблю путешествовать,У меня попугай,Я люблю мечтать
1502,Я оператор связи,У меня двое детей,Я хорошо готовлю,Я очень люблю походы,У меня в доме живет кошка
1503,Я проститутка,В разводе,Не люблю готовить,Ищу спонсора по жизни,Не люблю серьезные отношения


In [ ]:
L={}
alp=list('qwertyuiopasdfghjklzxcvbnm')
with open('data/TlkPersonaChatRus/TolokaPersonaChat_gk.jsonl', 'w') as out:
    with open('data/TlkPersonaChatRus/TolokaPersonaChat.jsonl', 'r') as inp:
        for line in inp:
            line = json.loads(line)
            dialog = line['dialog']
            for turn_i, turn_ in enumerate(dialog):
                turn = turn_['text'].split(' ')
                ngram=[]
                for word in turn:
                    word = [l for l in word if l.isalpha() and l not in alp]
                    word = ''.join(word)
                    word = morph.parse(word)[0]
                    if word.tag.POS=='NOUN':
                        ngram.append(word.normal_form)
                    elif word.tag.POS=='INFN':
                        ngram.append(word.normal_form)
                turn = ngram
                persona_ = line['persons'][turn_['person']]
                persona=[]
                for p in persona_:
                    p = p.split()
                    ngram=[]
                    for word in p:
                        word = [l for l in word if l.isalpha() and l not in alp]
                        word = ''.join(word)
                        word = morph.parse(word)[0]
                        if word.tag.POS=='NOUN':
                            ngram.append(word.normal_form)
                        elif word.tag.POS=='INFN':
                            ngram.append(word.normal_form)
                    persona.append(ngram)
                

                golden_p=set()
                for i, p in enumerate(persona):
                    for wp in p:
                        for wr in turn:
                            if wp in wr or wr in wp:
                                if i not in golden_p:
                                    golden_p.add(i)
                try:
                    L[len(golden_p)]+=1
                except:
                    L[len(golden_p)]=1
                dialog[turn_i]['gk'] = list(golden_p)
            

            line = json.dumps(line, ensure_ascii=False)
            out.write(line+'\n')
print(L)
            

In [3]:
with open('/home/stc/persona/data/TlkPersonaChatRus/TolokaPersonaChat_gk(test)v2.jsonl', 'w') as out:
    with open('/home/stc/persona/data/TlkPersonaChatRus/TolokaPersonaChat_gk(test).jsonl', 'r') as inp:
        for line in inp:
            line = json.loads(line)
            new_person_list = []
            for person in line['persons']:
                new_person = {'description':person, 'gender':'unknown'}
                new_person_list.append(new_person)
            line['persons'] = new_person_list
            line = json.dumps(line, ensure_ascii=False)
            out.write(line+'\n')